In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df=pd.read_csv('../input/hotel-bookings/hotel_bookings.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
# Replace missing values:
# agent: If no agency is given, booking was most likely made without one.
# company: If none given, it was most likely private.
# rest schould be self-explanatory.

In [ ]:
def data_clean(df):
    df.fillna(0,inplace=True)
    print(df.isnull().sum())

In [ ]:
data_clean(df)

In [ ]:
list=['children','adults','babies']

for i in list:
    print('{} has unique values as {}'.format(i,df[i].unique()))

In [ ]:
### seems to have some dirtiness in data as Adults,babies & children cant be zero at a same time

In [ ]:
df.shape

In [ ]:
len(df[df['adults']==0])

In [ ]:
filter=(df['children']==0) & (df['adults']==0) & (df['babies']==0)
df[filter]

In [ ]:
### Visualise Entire Dataframe where adult,children & babies are 0

In [ ]:
pd.set_option('display.max_columns',32)

In [ ]:
filter=(df['children']==0) & (df['adults']==0) & (df['babies']==0)
df[filter]

In [ ]:
data=df[~filter]

In [ ]:
data.shape

In [ ]:
data.head()

##  Where do the guests come from?Lets perform Spatial Analysis

In [ ]:
country_wise_data=data[data['is_canceled']==0]['country'].value_counts().reset_index()
country_wise_data.columns=['country','No of guests']
country_wise_data

In [ ]:
import folium
from folium.plugins import HeatMap

In [ ]:
basemap=folium.Map()

In [ ]:
country_wise_data.dtypes

In [ ]:
import plotly.express as px

In [ ]:
# show on map
map_guest = px.choropleth(country_wise_data,
                    locations=country_wise_data['country'],
                    color=country_wise_data['No of guests'], 
                    hover_name=country_wise_data['country'], 
                    title="Home country of guests")
map_guest.show()

#### People from all over the world are staying in these two hotels. Most guests are from Portugal and other countries in Europe

## How much do guests pay for a room per night?

In [ ]:
data.head()

#### Both hotels have different room types and different meal arrangements. Seasonal factors are also important. So the prices vary a lot. Since no currency information is given, but Portugal is part of the European Monetary Union, I assume that all prices are in EUR.

In [ ]:
data2=data[data['is_canceled']==0]

In [ ]:
# boxplot:
plt.figure(figsize=(12, 8))
sns.boxplot(x="reserved_room_type",
            y="adr",
            hue="hotel",
            data=data2)
plt.title("Price of room types per night and person", fontsize=16)
plt.xlabel("Room type", fontsize=16)
plt.ylabel("Price [EUR]", fontsize=16)
plt.legend(loc="upper right")
plt.ylim(0, 600)
plt.show()

#### This figure shows the average price per room, depending on its type and the standard deviation. Note that due to data anonymization rooms with the same type letter may not necessarily be the same across hotels.

## How does the price per night vary over the year?

In [ ]:
data_resort = data[(data["hotel"] == "Resort Hotel") & (data["is_canceled"] == 0)]
data_city = data[(data["hotel"] == "City Hotel") & (data["is_canceled"] == 0)]

In [ ]:
data_resort.head()

In [ ]:
resort_hotel=data_resort.groupby(['arrival_date_month'])['adr'].mean().reset_index()
resort_hotel

In [ ]:
city_hotel=data_city.groupby(['arrival_date_month'])['adr'].mean().reset_index()
city_hotel

In [ ]:
final=resort_hotel.merge(city_hotel,on='arrival_date_month')
final.columns=['month','price_for_resort','price_for_city_hotel']
final

#### now we will observe over here is month column is not in order, & if we will visualise we will get improper conclusion
#### so very first we have to provide right hierarchy to the month column

In [ ]:
!pip install sort-dataframeby-monthorweek

## Dependency package needs to be installed
!pip install sorted-months-weekdays

In [ ]:
import sort_dataframeby_monthorweek as sd

In [ ]:
def sort_data(df,colname):
    return sd.Sort_Dataframeby_Month(df,colname)


In [ ]:
final=sort_data(final,'month')
final

In [ ]:
px.line(final, x='month', y=['price_for_resort','price_for_city_hotel'], title='Room price per night over the Months')

### This clearly shows that the prices in the Resort hotel are much higher during the summer (no surprise here)., The price of the city hotel varies less and is most expensive during spring and autumn.

## Which are the most busy month or in which months Guests are high?

In [ ]:
data_resort.head()

In [ ]:
rush_resort=data_resort['arrival_date_month'].value_counts().reset_index()
rush_resort.columns=['month','no of guests']
rush_resort

In [ ]:
rush_city=data_city['arrival_date_month'].value_counts().reset_index()
rush_city.columns=['month','no of guests']
rush_city

In [ ]:
final_rush=rush_resort.merge(rush_city,on='month')
final_rush.columns=['month','no of guests in resort','no of guest in city hotel']
final_rush

In [ ]:
final_rush=sort_data(final_rush,'month')
final_rush

In [ ]:
final_rush.dtypes

In [ ]:
final_rush.columns

In [ ]:
px.line(data_frame=final_rush, x='month', y=['no of guests in resort','no of guest in city hotel'], title='Total no of guests per Months')

### Conclusion
     The City hotel has more guests during spring and autumn, when the prices are also highest.
    In July and August there are less visitors, although prices are lower.

    Guest numbers for the Resort hotel go down slighty from June to September, which is also when the prices are highest.
    Both hotels have the fewest guests during the winter.

## How long do people stay at the hotels?

In [ ]:
filter=data['is_canceled']==0
clean_data=data[filter]

In [ ]:
clean_data.head()

In [ ]:
clean_data["total_nights"] = clean_data["stays_in_weekend_nights"] + clean_data["stays_in_week_nights"]

In [ ]:
clean_data.head()

In [ ]:
stay=clean_data.groupby(['total_nights','hotel']).agg('count').reset_index()
stay=stay.iloc[:,0:3]
stay.head()

In [ ]:
stay=stay.rename(columns={'is_canceled':'Number of stays'})
stay.head()

In [ ]:
plt.figure(figsize=(20, 8))
sns.barplot(x = "total_nights", y = "Number of stays" , hue="hotel",
            hue_order = ["City Hotel", "Resort Hotel"], data=stay)

### Select important Features using Co-relation

In [ ]:
data.head()

In [ ]:
co_relation=data.corr()
co_relation

In [ ]:
co_relation=data.corr()["is_canceled"]
co_relation

In [ ]:
co_relation.abs().sort_values(ascending=False)

In [ ]:
co_relation.abs().sort_values(ascending=False)[1:]

In [ ]:
data.columns

    From this list it is apparent that lead_time, total_of_special_requests, required_car_parking_spaces, booking_changes and previous_cancellations are the 5 most important numerical features.
    However, to predict whether or not a booking will be canceled, the number of booking changes is a possible source of leakage, because this information can change over time.
    I will also not include days_in_waiting_list,booking changes  and arrival_date_year.

    The most important feature to exclude is the "reservation_status":

In [ ]:
data.groupby("is_canceled")["reservation_status"].value_counts()

In [ ]:
list_not=['days_in_waiting_list','arrival_date_year']

In [ ]:
num_features=[col for col in data.columns if data[col].dtype!='O' and col not in list_not]
num_features

In [ ]:
cat_not=['arrival_date_year', 'assigned_room_type', 'booking_changes', 'reservation_status', 'country','days_in_waiting_list']

In [ ]:
cat_features=[col for col in data.columns if data[col].dtype=='O' and col not in cat_not]
cat_features

In [ ]:
data_cat=data[cat_features]

In [ ]:
data_cat.head()

In [ ]:
import warnings
from warnings import filterwarnings
filterwarnings("ignore")

In [ ]:
data_cat['reservation_status_date']=pd.to_datetime(data_cat['reservation_status_date'])

In [ ]:
data_cat['year']=data_cat['reservation_status_date'].dt.year
data_cat['month']=data_cat['reservation_status_date'].dt.month
data_cat['day']=data_cat['reservation_status_date'].dt.day

In [ ]:
data_cat.head()

In [ ]:
data_cat.drop('reservation_status_date',axis=1,inplace=True)

In [ ]:
data_cat['cancellation']=data['is_canceled']

In [ ]:
data_cat.columns

### Feature Encoding

### Perform Mean Encoding Technique 

In [ ]:
cols=data_cat.columns[0:8]
cols

In [ ]:
for col in cols:
    print(data_cat.groupby([col])['cancellation'].mean())
    print('\n')

In [ ]:
for col in cols:
    print(data_cat.groupby([col])['cancellation'].mean().to_dict())
    print('\n')
    

In [ ]:
df=data_cat.copy()

In [ ]:

for col in cols:
    dict=data_cat.groupby([col])['cancellation'].mean().to_dict()
    data_cat[col]=data_cat[col].map(dict)
    

In [ ]:
data_cat.head(20)

In [ ]:
dataframe=pd.concat([data_cat,data[num_features]],axis=1)

In [ ]:
dataframe.head()

In [ ]:
dataframe.drop(['cancellation'],axis=1,inplace=True)

In [ ]:
dataframe.shape

### Handle Outliers

In [ ]:
sns.distplot(dataframe['lead_time'])

In [ ]:
import numpy as np

def handle_outlier(col):
    dataframe[col]=np.log1p(dataframe[col])


In [ ]:
handle_outlier('lead_time')

In [ ]:
sns.distplot(dataframe['lead_time'].dropna())

In [ ]:
sns.distplot(dataframe['adr'])

In [ ]:
handle_outlier('adr')

In [ ]:
sns.distplot(dataframe['adr'].dropna())

In [ ]:
dataframe.isnull().sum()

In [ ]:
dataframe.dropna(inplace=True)

In [ ]:
## separate dependent & independent features
y=dataframe['is_canceled']
x=dataframe.drop('is_canceled',axis=1)

### Feature Importance

In [ ]:

from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel


In [ ]:

# select a suitable alpha (equivalent of penalty).
# The bigger the alpha the less features that will be selected.

feature_sel_model = SelectFromModel(Lasso(alpha=0.005, random_state=0)) # remember to set the seed, the random state in this function
feature_sel_model.fit(x,y)

In [ ]:
feature_sel_model.get_support()

In [ ]:
cols=x.columns

In [ ]:
# let's print the number of total and selected features

# this is how we can make a list of the selected features
selected_feat = cols[(feature_sel_model.get_support())]

In [ ]:
# let's print some stats
print('total features: {}'.format((x.shape[1])))
print('selected features: {}'.format(len(selected_feat)))


In [ ]:
selected_feat

In [ ]:
x=x[selected_feat]

### splitting dataset & model Building

In [ ]:

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.75,random_state=0)

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression()
logreg.fit(x_train,y_train)

In [ ]:
y_pred=logreg.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
cm


In [ ]:
from sklearn.metrics import accuracy_score
score=accuracy_score(y_test,y_pred)
score


### Cross validate your model

In [ ]:
from sklearn.model_selection import cross_val_score
score=cross_val_score(logreg,x,y,cv=10)

In [ ]:
score

In [ ]:
score.mean()

### Play with multiple Algos

In [ ]:
#fit naive bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
### classifier models
models = []
models.append(('LogisticRegression', LogisticRegression()))
models.append(('Naive Bayes',GaussianNB()))
models.append(('RandomForest', RandomForestClassifier()))
models.append(('Decision Tree', DecisionTreeClassifier()))
models.append(('KNN', KNeighborsClassifier(n_neighbors = 5)))

In [ ]:


for name, model in models:
    print(name)
    model.fit(x_train, y_train)
    
    # Make predictions.
    predictions = model.predict(x_test)

    # Compute the error.
    from sklearn.metrics import confusion_matrix
    print(confusion_matrix(predictions, y_test))

    from sklearn.metrics import accuracy_score
    print(accuracy_score(predictions,y_test))
    print('\n')